This notebook is optional and serves to assist users in customizing the project's scope. It displays the table of contents structure, enabling users to identify and select specific sections they might want to exclude from the analysis. To exclude these sections, users should note the headings they want to omit and then specify these in the drop_headings variable within the 0_input.ipynb file, after reviewing the displayed table of contents. This step is optional; users who wish to include all sections in the report can skip this notebook and proceed directly to the 1b_preprocessing.ipynb file. The resulting table of contents will be printed at the end of this notebook.

Please ensure to paste the input_path, which is the location of 0_input.ipynb file, at the start of this notebook. This step is the only requirement to load all necessary information for the execution of the code.

Recommended Google Colab Runtime Type: CPU, as this notebook does not involve running machine learning models.

In [ ]:
# Input file path (must navigate at the beginning of each file)
input_path =  "/content/drive/My Drive/ImpactDataMining/Hurricane_Ian/Result"

All the below sections automatically retrieve data from the 0_input.ipynb file, as well as results from previous notebooks in this series. The code is designed to run using this information, so no further edits are required beyond this point.

In [ ]:
!pip install python-docx
!pip install anytree

import docx
import os
import json
import csv

from google.colab import drive
from anytree import Node, RenderTree, search

In [ ]:
import time
start_time = time.time()

In [ ]:
def current_path():
  print("Current working directory")
  print(os.getcwd())
  print()

current_path()
drive.mount('/content/drive')
os.chdir(input_path)
current_path()

Current working directory
/content/drive/MyDrive/ResilienceDataMining/Hurricane_Ian/Data

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Current working directory
/content/drive/My Drive/ResilienceDataMining/Hurricane_Ian/Result



In [ ]:
# Loading input from the JSON file
with open('0_input.json', 'r') as file:
    data = json.load(file)
    data_path = data['data_path']
    result_path = data['result_path']
    file_name = data['file_name']
    drop_headings = data['drop_headings']

In [ ]:
current_path()
drive.mount('/content/drive')
os.chdir(data_path)
current_path()

Current working directory
/content/drive/My Drive/ResilienceDataMining/Hurricane_Ian/Result

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Current working directory
/content/drive/My Drive/ResilienceDataMining/Hurricane_Ian/Data



In [ ]:
doc = docx.Document(file_name)

names = []
for para in doc.paragraphs:
    names.append(para.style.name)

text = []
for para in doc.paragraphs:
    text.append(para.text)

text_table = []
for table in doc.tables:
    table_data = []
    for row in table.rows:
        row_data = []
        for cell in row.cells:
            row_data.append(cell.text)
        table_data.append(row_data)
    text_table.append(table_data)

**Drop unnecessary sections**

In [ ]:
headings_tree = [(names[i], text[i]) for i, n in enumerate(names) if n.startswith('Heading')]
headings = [text[i] for i, n in enumerate(names) if n.startswith('Heading')]
headings_idx = [i for i, n in enumerate(names) if n.startswith('Heading')]

In [ ]:
# Root node
root = Node("Document")

# Keep track of the last node for each heading level
last_nodes = {0: root}

for level, title in headings_tree:
    level_num = int(level.split(' ')[-1])

    # Reset the last nodes for levels greater than the current level
    for i in range(level_num + 1, max(last_nodes.keys()) + 1):
        if i in last_nodes:
            del last_nodes[i]

    # Find the correct parent by looking at the last node at the previous level or above
    parent = None
    for i in range(level_num - 1, -1, -1):
        parent = last_nodes.get(i)
        if parent:
            break

    # Create the new node and store it as the last node for its level
    node = Node(title, parent=parent)
    last_nodes[level_num] = node

# Print the tree
print('TABLE OF CONTENTS')
for pre, _, node in RenderTree(root):
    print(f"{pre}{node.name}")

TABLE OF CONTENTS
Document
├── PREFACE
├── ATTRIBUTION GUIDANCE
│   ├── Reference to PVRR Analyses, Discussions or Recommendations
│   └── Citing Images from this PVRR
├── ACKNOWLEDGMENTS
├── TABLE OF CONTENTS
│   └── Common Terms & Acronyms
└── EXECUTIVE SUMMARY
    ├── Introduction
    │   ├── Loss of Life and Injuries
    │   ├── Official Response
    │   └── Report Scope
    ├── Hazard Characteristics
    │   ├── Meteorological Background
    │   ├── Wind Field
    │   ├── Storm Surge and Coastal Flooding
    │   ├── Rainfall and Inland Flooding
    │   ├── Tornadoes
    │   └── Comparative Case: Hurricane Charley (2004)
    ├── Local Codes and Construction Practices
    ├── Building Performance
    ├── Infrastructure Performance
    │   ├── Power Outages & Restoration
    │   │   └── Power Case Study: Solar Panels
    │   ├── Transportation Disruptions & Restoration
    │   │   └── Transportation Case Study: Sanibel Causeway
    │   └── Telecommunications
    ├── Coastal Protectiv

In [ ]:
end_time = time.time()
execution_time = end_time - start_time

print("Execution time:", execution_time, "seconds")

Execution time: 22.190863847732544 seconds
